# 확장 : 폴더로 받아들여서 진행하는 방식으로 데이터 받아봄.


In [1]:
!pip install tqdm

In [2]:
import os
import zipfile
import pandas as pd
import json
from tqdm import tqdm

def folder_to_csv(folder_path):

    # 처리된 데이터를 저장할 빈 데이터프레임 생성
    df_total = pd.DataFrame()

    # 주어진 폴더 내의 모든 ZIP 파일 찾기
    zip_files = [file for file in os.listdir(folder_path) if file.endswith('.zip')]

    # 각 ZIP 파일에 대해 반복 (tqdm 적용)
    for zip_file in tqdm(zip_files, desc="ZIP 파일 처리"):
        zip_file_path = os.path.join(folder_path, zip_file)
        
        # ZIP 파일 해제
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            # ZIP 파일 내부의 파일 목록을 얻어와서 tqdm에 적용
            zip_info_list = zip_ref.infolist()
            for file_info in tqdm(zip_info_list, desc="ZIP 파일 내 해제", leave=False):
                zip_ref.extract(file_info, path=folder_path)
            
            # 해제된 폴더 내 모든 JSON 파일 찾기
            json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
            
            # 각 JSON 파일에 대해 반복 (tqdm 적용)
            for json_file in tqdm(json_files, desc="JSON 파일 처리", leave=False):
                json_file_path = os.path.join(folder_path, json_file)
                
                # JSON 파일 읽기
                with open(json_file_path, 'r', encoding='utf-8') as file:
                    file_content = file.read()
                    file_content = file_content.replace('\n', '\\n').replace('\r', '\\r').replace('\t', '\\t')
                    try:
                        data = json.loads(file_content)
                    except json.JSONDecodeError as e:
                        continue  # JSON 오류가 있는 파일은 건너뛰고 다음 파일 처리
                                
                df = pd.json_normalize(data)  # JSON 데이터를 pandas 데이터프레임으로 변환
                

                # 전체 데이터프레임에 추가
                df_total = pd.concat([df_total, df], ignore_index=True, sort=False)
                
                # 처리가 끝난 JSON 파일 삭제 (선택적)
                os.remove(json_file_path)

    return df_total



In [3]:
train = folder_to_csv('C:\\data_shard\\라벨링 데이터\\train')



ZIP 파일 처리:   0%|          | 0/28 [00:00<?, ?it/s]

ZIP 파일 처리: 100%|██████████| 28/28 [1:08:32<00:00, 146.89s/it]


In [4]:
# 'train' 데이터프레임을 CSV 파일로 저장
train.to_csv('C:\\data_shard\\train_data.csv', index=False, encoding='utf-8-sig')

In [5]:
validation = folder_to_csv('C:\\data_shard\\라벨링 데이터\\validation')

ZIP 파일 처리: 100%|██████████| 56/56 [1:29:35<00:00, 96.00s/it]   


In [9]:


# 'validation' 데이터프레임을 CSV 파일로 저장
validation.to_csv('C:\\data_shard\\validation_data.csv', index=False, encoding='utf-8-sig')


In [2]:
import pandas as pd

In [4]:
train = pd.read_csv('train.csv',encoding='utf-8')

In [5]:
train.shape

(69641, 9)

In [ ]:
validation.shape


In [15]:
validation.head()

,version,dataSet.info.date,dataSet.info.occupation,dataSet.info.channel,dataSet.info.place,dataSet.info.gender,dataSet.info.ageRange,dataSet.info.experience,dataSet.question.raw.text,dataSet.question.raw.wordCount,...,rawDataInfo.question.channelCount,rawDataInfo.question.samplingRate,rawDataInfo.question.audioPath,rawDataInfo.answer.fileFormat,rawDataInfo.answer.fileSize,rawDataInfo.answer.duration,rawDataInfo.answer.samplingBit,rawDataInfo.answer.channelCount,rawDataInfo.answer.samplingRate,rawDataInfo.answer.audioPath
0,1.0,20230116,BM,MOCK,ONLINE,FEMALE,35-44,EXPERIENCED,요즘 논쟁이 많은 우버 택시 찬반에 대해서 논해 보시겠습니까,9,...,1,16kHz,/Mock/01.Management/Female/Experienced/ckmk_q_...,wav,2748238,85880,16,1,16kHz,/Mock/01.Management/Female/Experienced/ckmk_a_...
1,1.0,20230116,BM,MOCK,ONLINE,FEMALE,35-44,EXPERIENCED,지금 지원자분이 제일 가고 싶은 회사가 어딜까요,7,...,1,16kHz,/Mock/01.Management/Female/Experienced/ckmk_q_...,wav,2779598,86860,16,1,16kHz,/Mock/01.Management/Female/Experienced/ckmk_a_...
2,1.0,20230116,BM,MOCK,ONLINE,FEMALE,35-44,EXPERIENCED,면접자님은 이전 직장을 그만둔 특별한 이유가 있습니까 그리고 지금 다니는 직장도 같...,22,...,1,16kHz,/Mock/01.Management/Female/Experienced/ckmk_q_...,wav,2721998,85060,16,1,16kHz,/Mock/01.Management/Female/Experienced/ckmk_a_...
3,1.0,20230116,BM,MOCK,ONLINE,FEMALE,35-44,EXPERIENCED,요즘 공부하고 있는 분야가 있다면 무엇인지 한번 말씀을 해 보시고요 또 그동안에 본...,21,...,1,16kHz,/Mock/01.Management/Female/Experienced/ckmk_q_...,wav,2740558,85640,16,1,16kHz,/Mock/01.Management/Female/Experienced/ckmk_a_...
4,1.0,20230116,BM,MOCK,ONLINE,FEMALE,45-54,EXPERIENCED,일을 하게 되면 중국으로 출장을 갈 일이 조금 생길 수도 있습니다 그 빈도수도 굉장...,20,...,1,16kHz,/Mock/01.Management/Female/Experienced/ckmk_q_...,wav,1669198,52160,16,1,16kHz,/Mock/01.Management/Female/Experienced/ckmk_a_...


In [ ]:
train_use

In [7]:
train = train[['dataSet.info.occupation', 'dataSet.info.place', 'dataSet.info.gender','dataSet.info.ageRange','dataSet.info.experience', 'dataSet.question.raw.text', 'dataSet.answer.raw.text','dataSet.answer.intent','dataSet.answer.summary.text']]


In [8]:
train_cut = train

In [20]:
train.head()

,dataSet.info.occupation,dataSet.info.place,dataSet.info.gender,dataSet.info.ageRange,dataSet.info.experience,dataSet.question.raw.text,dataSet.answer.raw.text,dataSet.answer.intent,dataSet.answer.summary.text
0,BM,ONLINE,FEMALE,35-44,EXPERIENCED,요즘 논쟁이 많은 우버 택시 찬반에 대해서 논해 보시겠습니까,우버 택시에 대해서 말씀드려보겠습니다. 우버 택시 이제 바뀌는 현대사회에서 당연히 ...,"[{'text': '', 'expression': '', 'category': 'a...",우버 택시는 현대사회에서 당연히 나타날 부분이고 더 발전된 부분이 나타날 것이라 생...
1,BM,ONLINE,FEMALE,35-44,EXPERIENCED,지금 지원자분이 제일 가고 싶은 회사가 어딜까요,음 지금 제가 본인이 원했을 때 제가 가고 싶은 회사의 종류가 몇 가지가 있을 것 ...,"[{'text': '', 'expression': '', 'category': 'a...","저는 돈을 많이 주는 회사, 딱딱하고 엄격한 분위기 회사, 부드럽고 웃음이 많은 회..."
2,BM,ONLINE,FEMALE,35-44,EXPERIENCED,면접자님은 이전 직장을 그만둔 특별한 이유가 있습니까 그리고 지금 다니는 직장도 같...,아 이전 직장을 그만둔 계기에 대해서 말씀드리겠습니다. 어 이전 직장을 가장 그만두...,"[{'text': '', 'expression': '', 'category': 'a...",이전 직장을 그만둔 가장 큰 계기는 매너리즘이었습니다. 매너리즘에 빠지다 보니 몸이...
3,BM,ONLINE,FEMALE,35-44,EXPERIENCED,요즘 공부하고 있는 분야가 있다면 무엇인지 한번 말씀을 해 보시고요 또 그동안에 본...,네 요즘 공부하고 있는 것에 대해서 말씀드리겠습니다. 어 이제 경영 파트로 오게 되...,"[{'text': '', 'expression': '', 'category': 'a...","경영 파트로 오게 되면서 경영 회사의 원리를 알아야 겠다는 생각을 했고, 그러다 보..."
4,BM,ONLINE,FEMALE,45-54,EXPERIENCED,일을 하게 되면 중국으로 출장을 갈 일이 조금 생길 수도 있습니다 그 빈도수도 굉장...,당연히 해야죠. 저는 어 전에 했던 사업으로 인해서 중국에서의 경험이 조금 많습니다...,"[{'text': '', 'expression': '', 'category': 'a...",이 회사가 중국에 사업을 넓히는 것에 관심이 있다면 제 경험을 살려서 노하우를 공유...


In [21]:
train_cut.to_csv('C:\\data_shard\\train.csv', index=False, encoding='utf-8-sig')

In [21]:
make_query_train = train[['dataSet.info.occupation', 'dataSet.info.place', 'dataSet.info.gender','dataSet.info.ageRange','dataSet.info.experience', 'dataSet.question.raw.text','dataSet.answer.intent']]

In [22]:
make_query_train.head()

,dataSet.info.occupation,dataSet.info.place,dataSet.info.gender,dataSet.info.ageRange,dataSet.info.experience,dataSet.question.raw.text,dataSet.answer.intent
0,BM,ONLINE,FEMALE,35-44,EXPERIENCED,요즘 논쟁이 많은 우버 택시 찬반에 대해서 논해 보시겠습니까,"[{'text': '', 'expression': '', 'category': 'a..."
1,BM,ONLINE,FEMALE,35-44,EXPERIENCED,지금 지원자분이 제일 가고 싶은 회사가 어딜까요,"[{'text': '', 'expression': '', 'category': 'a..."
2,BM,ONLINE,FEMALE,35-44,EXPERIENCED,면접자님은 이전 직장을 그만둔 특별한 이유가 있습니까 그리고 지금 다니는 직장도 같...,"[{'text': '', 'expression': '', 'category': 'a..."
3,BM,ONLINE,FEMALE,35-44,EXPERIENCED,요즘 공부하고 있는 분야가 있다면 무엇인지 한번 말씀을 해 보시고요 또 그동안에 본...,"[{'text': '', 'expression': '', 'category': 'a..."
4,BM,ONLINE,FEMALE,45-54,EXPERIENCED,일을 하게 되면 중국으로 출장을 갈 일이 조금 생길 수도 있습니다 그 빈도수도 굉장...,"[{'text': '', 'expression': '', 'category': 'a..."


In [25]:
make_query_train['category'] = 0

C:\Users\r2com\AppData\Local\Temp\ipykernel_4392\2836748258.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  make_query_train['category'] = 0


In [26]:
make_query_train

,dataSet.info.occupation,dataSet.info.place,dataSet.info.gender,dataSet.info.ageRange,dataSet.info.experience,dataSet.question.raw.text,dataSet.answer.intent,category
0,BM,ONLINE,FEMALE,35-44,EXPERIENCED,요즘 논쟁이 많은 우버 택시 찬반에 대해서 논해 보시겠습니까,"[{'text': '', 'expression': '', 'category': 'a...",0
1,BM,ONLINE,FEMALE,35-44,EXPERIENCED,지금 지원자분이 제일 가고 싶은 회사가 어딜까요,"[{'text': '', 'expression': '', 'category': 'a...",0
2,BM,ONLINE,FEMALE,35-44,EXPERIENCED,면접자님은 이전 직장을 그만둔 특별한 이유가 있습니까 그리고 지금 다니는 직장도 같...,"[{'text': '', 'expression': '', 'category': 'a...",0
3,BM,ONLINE,FEMALE,35-44,EXPERIENCED,요즘 공부하고 있는 분야가 있다면 무엇인지 한번 말씀을 해 보시고요 또 그동안에 본...,"[{'text': '', 'expression': '', 'category': 'a...",0
4,BM,ONLINE,FEMALE,45-54,EXPERIENCED,일을 하게 되면 중국으로 출장을 갈 일이 조금 생길 수도 있습니다 그 빈도수도 굉장...,"[{'text': '', 'expression': '', 'category': 'a...",0
...,...,...,...,...,...,...,...,...
69636,MM,ONLINE,MALE,-34,NEW,지원자님께서 생각하시기에 본인 스스로 숫자와 창의 중에 어떤 것에 더 가깝다고 생각...,"[{'text': '', 'expression': '', 'category': 'a...",0
69637,MM,ONLINE,MALE,-34,NEW,지원자님께서 남들보다 더 우월한 그런 상위 일 퍼센트의 능력이 있으시다면 그러한 능...,"[{'text': '', 'expression': '', 'category': 'a...",0
69638,MM,ONLINE,MALE,-34,NEW,최근 고령화 사회로 가면서 침해로 인한 문제가 많이 대두되고 있습니다 이에 대해 건...,"[{'text': '', 'expression': '', 'category': 'a...",0
69639,MM,ONLINE,MALE,-34,NEW,일을 하다 보면 주말에도 근무를 해야 하거나 야근을 해야 하는 경우가 많이 발생을 ...,"[{'text': '', 'expression': '', 'category': 'a...",0


In [44]:
import ast


In [ ]:
make_query_train['intent'] = 0

In [ ]:
for index in range(len(make_query_train['dataSet.answer.intent'])):
    make_query_train['intent'][index] = ast.literal_eval(make_query_train['dataSet.answer.intent'][index])[0]['category']

In [49]:
make_query_train['case'] = 0

In [ ]:
for index in range(len(make_query_train)):
    if make_query_train['intent'][index] in ["technology","attitude"]:
        make_query_train['case'][index] = 'job'
    elif make_query_train['intent'][index] in ["background", "personality" ,"etc"]:
        make_query_train['case'][index] = 'personality'

In [53]:
make_query_train.head()

,dataSet.info.occupation,dataSet.info.place,dataSet.info.gender,dataSet.info.ageRange,dataSet.info.experience,dataSet.question.raw.text,dataSet.answer.intent,category,case
0,BM,ONLINE,FEMALE,35-44,EXPERIENCED,요즘 논쟁이 많은 우버 택시 찬반에 대해서 논해 보시겠습니까,attitude,0,job
1,BM,ONLINE,FEMALE,35-44,EXPERIENCED,지금 지원자분이 제일 가고 싶은 회사가 어딜까요,attitude,0,job
2,BM,ONLINE,FEMALE,35-44,EXPERIENCED,면접자님은 이전 직장을 그만둔 특별한 이유가 있습니까 그리고 지금 다니는 직장도 같...,attitude,0,job
3,BM,ONLINE,FEMALE,35-44,EXPERIENCED,요즘 공부하고 있는 분야가 있다면 무엇인지 한번 말씀을 해 보시고요 또 그동안에 본...,attitude,0,job
4,BM,ONLINE,FEMALE,45-54,EXPERIENCED,일을 하게 되면 중국으로 출장을 갈 일이 조금 생길 수도 있습니다 그 빈도수도 굉장...,attitude,0,job


In [ ]:
for idx in range(len(make_query_train)):
    make_query_train['dataSet.info.ageRange'][index] = make_query_train['dataSet.info.ageRange'][-2:]